<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9f38d1c262f173325ee59fb1ba6da638185273cb04c0fdfe6a6ae8573374219e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-11 09:57:13
-------------------
qualified stocks: 86
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  99.50 K
Current:  1.42 C
-------------------
Today PnL: -54.91 K (-0.38%)
Current PnL: -22.74 L (-14.9%)
CY Booked + Current PnL: -9.55 L (-6.26%)
-------------------
Total profit:  1.53 L
Total loss:  -24.27 L
-------------------
Total Booked + Current PnL: 17.43 L (13.88%)
Total Booked PnL: 40.17 L (31.98%)
Curr Year Booked PnL: 13.19 L (9.26%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 90.20 L (63.35%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,101.49,51.0,M-SC,2.20,86807.0,-13970.0,14063.0,0.30,-13.86,16.20,0.09,245.0,-0.99,0.61,14.05,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.25,47.0,X-MC,3.15,88710.0,-1968.0,16873.0,0.22,-2.17,19.02,16.44,101.0,-0.12,0.63,17.57,X40,ATH,PHARMA
42,ITC,452.00,-38.34,46.0,X-LC,0.83,198876.0,-1262.0,22155.0,0.28,-0.63,11.14,10.44,4.0,-0.06,1.41,4.90,X40,NTT,FMCG
34,ICICIGI,2252.93,-13.84,55.0,X-MC,6.61,186316.0,11996.0,23215.0,-1.09,6.88,12.46,20.19,91.0,0.52,1.32,23.04,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-3.90,58.0,X-LC,8.21,295589.0,30163.0,26633.0,0.02,11.36,9.01,21.40,11.0,1.13,2.09,18.76,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,680.0,-16.36,61.0,X-MC,1.17,224775.0,-2564.0,50619.0,3.62,-1.13,22.52,21.14,106.0,-0.05,1.59,26.69,XY24,NTT,PAINTS
39,INDUSINDBK,1800.0,278.85,68.0,L-MC,7.40,46523.0,-30684.0,56074.0,2.11,-39.74,120.53,32.89,258.0,-0.55,0.33,28.12,XR,NTT,BANKS
38,INDIGOPNTS,1408.0,121.54,75.0,M-SC,5.37,145452.0,-29107.0,29134.0,1.53,-16.67,20.03,0.02,221.0,-1.00,1.03,26.52,OX40N,NTT,PAINTS
74,TATAMOTORS,1065.0,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
17,CERA,9475.0,-24.58,43.0,H-SC,1.78,137666.0,-38237.0,80259.0,1.05,-21.74,58.30,23.89,149.0,-0.48,0.97,18.89,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VAIBHAVGBL,521.00,65.84,55.0,H-SC,5.57,143820.0,-38955.0,150019.0,-2.75,-21.31,104.31,60.77,125.0,-0.26,1.02,29.52,XR,NTT,JEWELLERY
84,WHIRLPOOL,2270.00,-44.33,39.0,M-SC,8.42,115405.0,607.0,93432.0,-2.40,0.53,80.96,81.92,223.0,0.01,0.82,36.62,XR,NTT,DURABLES
60,ROUTE,2227.21,-58.98,21.0,H-SC,23.28,67463.0,-65603.0,157486.0,-2.12,-49.30,233.44,69.05,118.0,-0.42,0.48,0.00,SR,ATH,IT
26,GREENPANEL,537.00,199.10,38.0,M-SC,5.26,133444.0,-49634.0,128613.0,-2.02,-27.11,96.38,43.14,230.0,-0.39,0.94,25.52,XY24,NTT,MISC
59,REPCOHOME,880.00,-54.55,56.0,H-SC,8.68,259869.0,-26636.0,283959.0,-1.94,-9.30,109.27,89.82,134.0,-0.09,1.84,37.49,XY24,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-44.33,39.0,M-SC,8.42,115405.0,607.0,93432.0,-2.4,0.53,80.96,81.92,223.0,0.01,0.82,36.62,XR,NTT,DURABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.49,51.0,M-SC,2.20,86807.0,-13970.0,14063.0,0.30,-13.86,16.20,0.09,245.0,-0.99,0.61,14.05,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.0,121.54,75.0,M-SC,5.37,145452.0,-29107.0,29134.0,1.53,-16.67,20.03,0.02,221.0,-1.00,1.03,26.52,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.16,40.0,H-SC,2.26,219312.0,-50355.0,86694.0,-0.87,-18.67,39.53,13.47,138.0,-0.58,1.55,11.51,XY24,NTT,PAINTS
17,CERA,9475.0,-24.58,43.0,H-SC,1.78,137666.0,-38237.0,80259.0,1.05,-21.74,58.30,23.89,149.0,-0.48,0.97,18.89,OX40N,NTT,CERAMICS
43,JCHAC,2282.0,19943.48,63.0,M-SC,1.49,97927.0,-29878.0,29868.0,0.02,-23.38,30.50,-0.01,232.0,-1.00,0.69,10.89,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,55.57,65.0,M-MC,10.42,245282.0,20320.0,147341.0,-1.32,9.03,60.07,74.53,192.0,0.14,1.73,44.20,XY24,BTT,STEEL
32,HINDZINC,730.22,29.16,54.0,M-LC,9.18,211018.0,5942.0,106627.0,0.10,2.90,50.53,54.89,52.0,0.06,1.49,27.69,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-44.33,39.0,M-SC,8.42,115405.0,607.0,93432.0,-2.40,0.53,80.96,81.92,223.0,0.01,0.82,36.62,XR,NTT,DURABLES
37,INDIAMART,4810.62,-53.71,50.0,H-SC,5.07,129140.0,5804.0,125821.0,-0.62,4.71,97.43,106.72,119.0,0.05,0.91,29.18,AR,ATH,MISC
62,SAMMAANCAP,326.00,-193.18,56.0,M-SC,35.22,160740.0,11520.0,132659.0,-0.36,7.72,82.53,96.62,208.0,0.09,1.14,73.96,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,29.16,54.0,M-LC,9.18,211018.0,5942.0,106627.0,0.10,2.90,50.53,54.89,52.0,0.06,1.49,27.69,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.71,50.0,H-SC,5.07,129140.0,5804.0,125821.0,-0.62,4.71,97.43,106.72,119.0,0.05,0.91,29.18,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-44.33,39.0,M-SC,8.42,115405.0,607.0,93432.0,-2.40,0.53,80.96,81.92,223.0,0.01,0.82,36.62,XR,NTT,DURABLES
85,WIPRO,420.00,-15.61,45.0,M-LC,5.71,148478.0,-2467.0,111923.0,-0.15,-1.63,75.38,72.51,53.0,-0.02,1.05,4.29,XR,NTT,IT
47,KPIGREEN,731.05,2.88,44.0,H-SC,9.78,122472.0,-2825.0,61750.0,0.35,-2.25,50.42,47.03,141.0,-0.05,0.87,53.35,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-18.48,17.0,X-MC,5.53,250335.0,-3561.0,85865.0,-1.55,-1.40,34.30,32.42,105.0,-0.04,1.77,13.48,X40,ATH,FMCG
45,JSWINFRA,342.00,-25.18,24.0,X-MC,4.46,187515.0,-12832.0,43335.0,-0.36,-6.40,23.11,15.23,178.0,-0.30,1.33,23.78,X40N,NTT,REALTY
74,TATAMOTORS,1065.00,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.50,86876.0,-41794.0,87093.0,-0.27,-32.48,100.25,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
23,DMART,5391.45,-20.74,29.0,X-LC,6.77,202005.0,-8402.0,67571.0,0.53,-3.99,33.45,28.12,38.0,-0.12,1.43,18.69,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,17907.65,-31.74,36.0,X-MC,0.65,197715.0,-3105.0,70901.0,-0.78,-1.55,35.86,33.76,80.0,-0.04,1.40,3.57,X40,ATH,FMCG
42,ITC,452.00,-38.34,46.0,X-LC,0.83,198876.0,-1262.0,22155.0,0.28,-0.63,11.14,10.44,4.0,-0.06,1.41,4.90,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.39,39.0,H-LC,1.61,151950.0,-34145.0,81628.0,-0.50,-18.35,53.72,25.51,15.0,-0.42,1.07,11.98,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-9.58,34.0,X-MC,3.34,301560.0,-26431.0,132958.0,-1.49,-8.06,44.09,32.48,92.0,-0.20,2.13,2.45,X40,ATH,ELECTRICAL
79,UNITDSPR,1644.00,-6.95,60.0,X-LC,3.61,244034.0,9890.0,38728.0,0.66,4.22,15.87,20.77,86.0,0.26,1.73,11.43,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.50,86876.0,-41794.0,87093.0,-0.27,-32.48,100.25,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
54,QUESS,986.00,-51.52,32.0,X-SC,37.26,48114.0,-16892.0,167817.0,-1.88,-25.99,348.79,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
52,PAGEIND,51769.93,-29.48,36.0,X-MC,7.66,159460.0,-4280.0,47615.0,-0.83,-2.61,29.86,26.47,82.0,-0.09,1.13,1.03,X40,ATH,APPARELS
20,COLPAL,3726.84,-7.61,38.0,X-MC,7.15,215730.0,-47635.0,156944.0,-0.64,-18.09,72.75,41.51,84.0,-0.30,1.53,1.24,XY25,ATH,FMCG
10,BAJAJHFL,181.50,-16.38,35.0,X-MC,6.37,176070.0,-24436.0,121594.0,-0.83,-12.19,69.06,48.45,90.0,-0.20,1.25,1.53,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-51.52,32.0,X-SC,37.26,48114.0,-16892.0,167817.0,-1.88,-25.99,348.79,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-41.04,39.0,X-SC,4.05,80379.0,-64781.0,140711.0,-0.09,-44.63,175.06,52.31,136.0,-0.46,0.57,7.59,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.50,86876.0,-41794.0,87093.0,-0.27,-32.48,100.25,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.25,47.0,X-MC,3.15,88710.0,-1968.0,16873.0,0.22,-2.17,19.02,16.44,101.0,-0.12,0.63,17.57,X40,ATH,PHARMA
33,HONAUT,58357.33,-23.74,39.0,X-SC,0.89,107415.0,-20523.0,67661.0,-0.58,-16.04,62.99,36.84,143.0,-0.30,0.76,9.97,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.36,52.0,X-LC,9.36,285243.0,-60713.0,127418.0,0.31,-17.55,44.67,19.28,1.0,-0.48,2.02,5.45,X40,ATH,IT
40,INFY,2275.00,-17.09,61.0,X-LC,4.72,322887.0,9999.0,161702.0,0.16,3.20,50.08,54.87,3.0,0.06,2.28,11.74,X40,BTT,IT
42,ITC,452.00,-38.34,46.0,X-LC,0.83,198876.0,-1262.0,22155.0,0.28,-0.63,11.14,10.44,4.0,-0.06,1.41,4.90,X40,NTT,FMCG
82,VBL,671.64,-17.63,50.0,X-LC,5.76,296542.0,-19300.0,131961.0,0.33,-6.11,44.50,35.67,5.0,-0.15,2.10,6.90,X40N,ATH,FMCG
1,ABB,7934.00,-42.88,32.0,H-LC,4.99,238608.0,-23011.0,142234.0,-0.42,-8.80,59.61,45.57,7.0,-0.16,1.69,1.53,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,27.23,76666.0,-36883.0,76934.0,-1.75,-32.48,100.35,35.27,268.0,-0.48,0.54,87.64,XR,NTT,HOTELS
7,ASIANTILES,137.00,7466.67,64.0,L-SC,13.99,83049.0,-10761.0,87376.0,-1.23,-11.47,105.21,81.67,269.0,-0.12,0.59,60.83,XR,NTT,CERAMICS
50,MASFIN,398.61,-20.05,46.0,H-SC,7.91,90855.0,-7125.0,28728.0,-0.70,-7.27,31.62,22.05,152.0,-0.25,0.64,32.06,XR,ATH,FINANCE
47,KPIGREEN,731.05,2.88,44.0,H-SC,9.78,122472.0,-2825.0,61750.0,0.35,-2.25,50.42,47.03,141.0,-0.05,0.87,53.35,MH,ATH,POWER
80,VAIBHAVGBL,521.00,65.84,55.0,H-SC,5.57,143820.0,-38955.0,150019.0,-2.75,-21.31,104.31,60.77,125.0,-0.26,1.02,29.52,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7466.67,64.0,L-SC,13.99,83049.0,-10761.0,87376.0,-1.23,-11.47,105.21,81.67,269.0,-0.12,0.59,60.83,XR,NTT,CERAMICS
80,VAIBHAVGBL,521.00,65.84,55.0,H-SC,5.57,143820.0,-38955.0,150019.0,-2.75,-21.31,104.31,60.77,125.0,-0.26,1.02,29.52,XR,NTT,JEWELLERY
6,ASIANPAINT,3465.66,-7.96,72.0,X-LC,6.15,232311.0,-19457.0,72667.0,-0.40,-7.73,31.28,21.13,27.0,-0.27,1.64,24.96,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,121.54,75.0,M-SC,5.37,145452.0,-29107.0,29134.0,1.53,-16.67,20.03,0.02,221.0,-1.00,1.03,26.52,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,21.94,67.0,X-SC,6.66,215660.0,24654.0,33039.0,0.91,12.91,15.32,30.21,157.0,0.75,1.53,36.48,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.79
1,25,44.79
2,50,76.19


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.50
LC    30.48
MC    26.04
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.75
X40      19.34
X40N     12.47
XY25     10.26
XR        9.69
AR        7.72
OX40N     6.61
X200      1.80
X5K       1.49
SR        1.02
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    21.66
X-LC    20.99
M-SC    12.53
X-SC     5.58
H-LC     4.56
M-LC     3.94
H-MC     2.32
M-MC     1.73
L-SC     1.46
L-LC     0.99
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.04,-4.58,38.39
IT,12.59,-18.11,81.34
FINANCE,12.12,-10.52,60.61
MISC,7.68,-19.37,79.58
PAINTS,5.81,-12.35,29.09
ELECTRICAL,5.65,-13.02,53.09
INSURANCE,4.49,0.30,34.84
BANKS,2.73,-38.57,128.87
AUTO,2.71,-49.99,112.59


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3322224.0,21
XR,1287173.0,13
AR,1194910.0,9
X40,957622.0,13
X40N,708019.0,9
OX40N,549157.0,9
XY25,485281.0,7
SR,277285.0,2
X5K,106627.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3447923.0,24
X-MC,1349461.0,15
M-SC,1272993.0,15
X-LC,1072418.0,12
X-SC,559740.0,6
H-LC,294063.0,3
M-LC,266536.0,3
H-MC,259933.0,2
L-SC,254857.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1196142.0      6
           AR         845435.0      5
           XR         811735.0      7
M-SC       XY24       743769.0      6
X-MC       XY24       568285.0      4
X-LC       X40        442523.0      5
X-MC       X40        428006.0      7
X-LC       XY24       314150.0      2
X-SC       X40N       304830.0      4
H-SC       SR         277285.0      2
           OX40N      255576.0      3
X-LC       X40N       238260.0      3
H-LC       AR         223862.0      2
X-MC       XY25       188241.0      2
H-MC       XY24       184720.0      1
X-SC       XY24       167817.0      1
X-MC       X40N       164929.0      2
L-SC       XR         164310.0      2
M-MC       XY24       147341.0      1
M-SC       XR         143131.0      2
           XY25       132659.0      1
           OX40N      127821.0      4
           AR         125613.0      2
M-LC       XR         111923.0      1
           X5K        106627.0      1
L-SC       OX40N       90547.0      1
X-SC       X40         87093.0      1
X-LC       XY25        77485.0      2
H-MC       OX40N       75213.0      1
H-LC       X200        70201.0      1
H-SC       MH          61750.0      1
L-MC       XR          56074.0      1
M-LC       XY25        47986.0      1
L-LC       XY25        38910.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
